# 表计模型训练教程
1. 环境配置
2. 数据集下载与上传
3. 修改配置文件并训练
4. 导出onnx及ncnn

## 1. 环境配置
1. 下载[edgelab](https://github.com/Seeed-Studio/Edgelab)仓库代码
2. 运行项目环境配置脚本[env_config.py](https://github.com/Seeed-Studio/Edgelab/blob/master/tools/env_config.py)

首先切换到用户的主目录，开始下载仓库代码。

In [1]:
%cd ~
!git clone https://github.com/Seeed-Studio/Edgelab
%cd edgelab

/root
Cloning into 'edgelab'...
remote: Enumerating objects: 2359, done.
remote: Total 2359 (delta 0), reused 0 (delta 0), pack-reused 2359
Receiving objects: 100% (2359/2359), 1.71 GiB | 17.43 MiB/s, done.
Resolving deltas: 100% (233/233), done.
/root/edgelab


仓库代码下载完成后可直接运行环境配置脚本。

In [2]:
!python tools/env_config.py

----------ensure base env----------

    check python, root, pytorch version, auto install these binary:

    * make
    * g++
    * git
    * wget
    * unzip
    * opencv
    * mmcv (not compulsory)
    
/usr/bin/python3: No module named ensurepip
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Reading package lists... Done
Building dependency tree       
Reading state information... Done
pkg-config is already the newest version (0.29.1-0ubuntu2).
cmake is already the newest version (3.10.2-1ubuntu2.18.04.2).
curl is already the newest version (7.58.0-2ubuntu3.21).
git is already the newest version (1:2.17.1-1ubuntu0.13).
python3-dev is already the newest version (3.6.7-1~18.04).
python3-dev set to manually installed.
wget is already the newest version (1.19.4-1ubuntu2.2).
libavcodec-dev is already the newest version (7:3.4.11-0ubuntu0.1).
libavcodec-dev set to manually installed.
libavformat-dev is already the newest version (7:3.4.

在相关环境配置完成后需要将本项目的路径配置到`PYTHONPATH`环境变量中：

In [3]:
import os
home = os.environ['HOME']
os.environ['PYTHONPATH'] = f'{home}/edgelab'

## 2. 数据集下载与上传
1. 首先下载[表计数据集](https://1drv.ms/u/s!AqG2uRmVUhlShtIhyd_7APHXEhpeXg?e=WwGx5m)
2. 将数据集上传至用户根目录

In [6]:
%mkdir -p ~/datasets
!unzip -q -n ~/meter.zip -d ~/datasets

## 3. 训练表计模型
此时可直接进行训练，只需运行`tools/train.py`程序即可。

在运行命令中需要指定任务类别(`mmpose`)、模型配置文件路径(`./configs/pfld/pfld_mv2n_112.py`)。更多参数可查看[tools/train.py](https://github.com/Seeed-Studio/Edgelab/blob/274df3d7f4670260f38f1ef55658d2090266d060/tools/train.py#L18)文件。

In [10]:
!~/anaconda3/envs/edgelab/bin/python tools/train.py mmpose configs/pfld/pfld_mv2n_112.py  --cfg-options total_epochs=30

/root/anaconda3/envs/edgelab/lib/python3.8/site-packages/mmcv/__init__.py:20: UserWarning: On January 1, 2023, MMCV will release v2.0.0, in which it will remove components related to the training process and add a data transformation module. In addition, it will rename the package names mmcv to mmcv-lite and mmcv-full to mmcv. See https://github.com/open-mmlab/mmcv/blob/master/docs/en/compatibility.md for more details.
  warnings.warn(
/root/anaconda3/envs/edgelab/lib/python3.8/site-packages/mmpose/utils/setup_env.py:32: UserWarning: Setting OMP_NUM_THREADS environment variable for each process to be 1 in default, to avoid your system being overloaded, please further tune the variable for optimal performance in your application as needed.
  warnings.warn(
/root/anaconda3/envs/edgelab/lib/python3.8/site-packages/mmpose/utils/setup_env.py:42: UserWarning: Setting MKL_NUM_THREADS environment variable for each process to be 1 in default, to avoid your system being overloaded, please furthe

## 4. 导出ONNX及NCNN
1. 使用torch2onnx.py转换刚才完成训练的权重文件
2. onnx文件生成位置位于原权重文件相同位置(相同文件名，后缀.onnx)

在从pth导出到onnx会有一个专用的脚本导出，而导出ncnn只能使用另一个转换脚本从onnx转到ncnn，所以想要ncnn格式的模型权重，需要先将模型从pth转到onnx。
对于onnx的导出很简单，只需一行命令即可：

In [14]:
!~/anaconda3/envs/edgelab/bin/python tools/torch2onnx.py --config configs/pfld/pfld_mv2n_112.py --checkpoint work_dirs/pfld_mv2n_112/exp1/latest.pth --task mmpose --shape 112

/root/anaconda3/envs/edgelab/lib/python3.8/site-packages/mmcv/__init__.py:20: UserWarning: On January 1, 2023, MMCV will release v2.0.0, in which it will remove components related to the training process and add a data transformation module. In addition, it will rename the package names mmcv to mmcv-lite and mmcv-full to mmcv. See https://github.com/open-mmlab/mmcv/blob/master/docs/en/compatibility.md for more details.
  warnings.warn(
load checkpoint from local path: work_dirs/pfld_mv2n_112/exp1/latest.pth
/root/anaconda3/envs/edgelab/lib/python3.8/site-packages/mmcv/onnx/symbolic.py:481: UserWarning: DeprecationWarning: This function will be deprecated in future. Welcome to use the unified model deployment toolbox MMDeploy: https://github.com/open-mmlab/mmdeploy
  warnings.warn(msg)
Successfully exported ONNX model: /root/edgelab/work_dirs/pfld_mv2n_112/exp1/latest.onnx



使用[tools/export_quantize.py](https://github.com/Seeed-Studio/Edgelab/blob/master/tools/export_quantize.py)脚本可将生成的onnx文件转为ncnn或进行量化

onnx转ncnn命令如下：

In [18]:
!~/anaconda3/envs/edgelab/bin/python tools/export_quantize.py --onnx work_dirs/pfld_mv2n_112/exp1/latest.onnx --type ncnn

2022-12-01 11:09:29,418 - ENV_CONFIG - INFO - ncnn export succeeded!
